## Imports and Data

In [1]:
import re, pandas as pd, matplotlib.pyplot as plt, numpy as np
from sklearn.feature_extraction.text import CountVectorizer # LDA uses term frequencies
from sklearn.decomposition import LatentDirichletAllocation # LDA we are going to use

In [14]:
df = pd.read_csv('../output/TEDall_speakers.csv', index_col=0)
print(df.shape)
df.head()

(1747, 26)


,Talk_ID,public_url,headline,description,event,duration,published,tags,views,text,...,speaker2_introduction,speaker2_profile,speaker_3,speaker3_occupation,speaker3_introduction,speaker3_profile,speaker_4,speaker4_occupation,speaker4_introduction,speaker4_profile
Set,,,,,,,,,,,,,,,,,,,,,
only,1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
only,7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
only,53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
only,66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
only,92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Create list of just the texts of the talks
texts = df.text.tolist()

## Topic Modeling

RG thread on [What is a good way to perform topic modeling on short text?](https://www.researchgate.net/post/What_is_a_good_way_to_perform_topic_modeling_on_short_text)

### Notes on Parameters

* **Alpha** parameter is Dirichlet prior concentration parameter that represents document-topic density — with a higher alpha, documents are assumed to be made up of more topics and result in more specific topic distribution per document.
* **Beta** parameter is the same prior concentration parameter that represents topic-word density — with high beta, topics are assumed to made of up most of the words and result in a more specific word distribution per topic.

[Topic Modeling in Python: Latent Dirichlet Allocation (LDA)](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0)

In [17]:
# Create vectorizer: pass parameters here
vectorizer = CountVectorizer()

In [19]:
tf_all = vectorizer.fit_transform(texts)
tf_all.shape

(1747, 50379)

We are not taking much off the top end with the code below: a term has to occur in 100% of documents to be ignored. At the bottom end, we are simply taking away terms that occur across fewer than three documents. (If we increase it to 5, we love an additional 6000 words.)

In [25]:
vec_fewer = CountVectorizer(max_df = 1.0,
                                min_df = 3)

In [26]:
tf_fewer = vec_fewer.fit_transform(texts)
tf_fewer.shape

(1747, 22719)

In [29]:
# LDA needs to be told how many topics to expect:
n_topics = 100

In [30]:
# First we configure our LDA-izer:
lda = LatentDirichletAllocation(n_components = n_topics, 
                                max_iter = 20, 
                                learning_method = 'online', 
                                learning_offset = 50.,
                                random_state = 0)

In [32]:
lda.fit(tf_fewer)

display_topics(lda, tf_feature_names, n_top_words)
# Get W (DTM) and H (WTM) arrays
lda_W = lda.transform(tf)
lda_H = lda.components_

df_lda_DTM = pd.DataFrame(data= lda_W, index = docs, columns = topic_labels)
# df_lda_DTM.to_csv('../outputs/lda_DTM.csv', sep=',')
print(df_lda_DTM)

NameError: name 'display_topics' is not defined